CNN

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from keras.callbacks import EarlyStopping 
from keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
df= pd.read_csv("Embedded_data.csv")
train = df.drop(['Tokens'], axis = 1)

X = train.Text
Y = train.Label
le = LabelEncoder() 
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
X_train = X_train.astype(str)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = utils.pad_sequences(sequences,maxlen=max_len)

def CNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 1000, input_length=max_len)(inputs)
    layer = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Flatten()(layer)
    layer = Dense(512, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model
model = CNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

X_test = X_test.astype(str).tolist()

# Now process X_test with the tokenizer and pad_sequences
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences, maxlen=max_len)

# Evaluate the model on the test data
accr = model.evaluate(test_sequences_matrix, Y_test)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

# Predict probabilities using the model
y_pred_proba = model.predict(test_sequences_matrix)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate and print evaluation metrics
print("Precision:", precision_score(Y_test, y_pred))
print("Recall:", recall_score(Y_test, y_pred))
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(Y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, y_pred))


print(' Loss: {:0.3f}\n'.format(accr[0],accr[1]))


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 200, 1000)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 200, 512)       │     1,536,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 100, 128)       │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 512)            │     3,277,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_layer (Dense)               │ (None, 1)              │           513 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,011,073 (22.93 MB)

 Trainable params: 6,011,073 (22.93 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.5840 - loss: 0.6903 - val_accuracy: 0.5899 - val_loss: 0.6718
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.6144 - loss: 0.6800 - val_accuracy: 0.6631 - val_loss: 0.6134
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.6977 - loss: 0.5710 - val_accuracy: 0.6574 - val_loss: 0.6065
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.7503 - loss: 0.5038 - val_accuracy: 0.6928 - val_loss: 0.5963
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8048 - loss: 0.4215 - val_accuracy: 0.6905 - val_loss: 0.6508
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.6836 - loss: 0.6103
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - accuracy: 0.6836 - loss: 0.6103
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step
Precision: 0.5848214285714286
Recall: 0.6285988483685221
Accuracy: 0.6772727272727272
ROC-AUC Score: 0.6688050562243111
Confusion Matrix:
 [[1133  465]
 [ 387  655]]
 Loss: 0.64

changing layers(256)


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from keras.callbacks import EarlyStopping 
from keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
df= pd.read_csv("Embedded_data.csv")
train = df.drop(['Tokens'], axis = 1)

X = train.Text
Y = train.Label
le = LabelEncoder() 
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
X_train = X_train.astype(str)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = utils.pad_sequences(sequences,maxlen=max_len)

def CNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 1000, input_length=max_len)(inputs)
    layer = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Flatten()(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

X_test = X_test.astype(str).tolist()

# Now process X_test with the tokenizer and pad_sequences
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences, maxlen=max_len)

# Evaluate the model on the test data
accr = model.evaluate(test_sequences_matrix, Y_test)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

# Predict probabilities using the model
y_pred_proba = model.predict(test_sequences_matrix)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate and print evaluation metrics
print("Precision:", precision_score(Y_test, y_pred))
print("Recall:", recall_score(Y_test, y_pred))
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(Y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, y_pred))


print(' Loss: {:0.3f}\n'.format(accr[0],accr[1]))

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.6442 - loss: 0.6468 - val_accuracy: 0.7172 - val_loss: 0.5543
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.7619 - loss: 0.4785 - val_accuracy: 0.7223 - val_loss: 0.5537
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.8381 - loss: 0.3567 - val_accuracy: 0.6965 - val_loss: 0.6526
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.6810 - loss: 0.6610
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - accuracy: 0.6810 - loss: 0.6610
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step
Precision: 0.6323024054982818
Recall: 0.5354025218234724
Accuracy: 0.696969696969697
ROC-AUC Score: 0.6679498625276467
Confusion Matrix:
 [[1288  321]
 [ 479  552]]
 Loss: 0.649



changing layers (128)

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from keras.callbacks import EarlyStopping 
from keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
df= pd.read_csv("Embedded_data.csv")
train = df.drop(['Tokens'], axis = 1)

X = train.Text
Y = train.Label
le = LabelEncoder() 
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
X_train = X_train.astype(str)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = utils.pad_sequences(sequences,maxlen=max_len)

def CNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 1000, input_length=max_len)(inputs)
    layer = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Flatten()(layer)
    layer = Dense(128, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

X_test = X_test.astype(str).tolist()

# Now process X_test with the tokenizer and pad_sequences
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences, maxlen=max_len)

# Evaluate the model on the test data
accr = model.evaluate(test_sequences_matrix, Y_test)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

# Predict probabilities using the model
y_pred_proba = model.predict(test_sequences_matrix)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate and print evaluation metrics
print("Precision:", precision_score(Y_test, y_pred))
print("Recall:", recall_score(Y_test, y_pred))
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(Y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, y_pred))


print(' Loss: {:0.3f}\n'.format(accr[0],accr[1]))

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.6664 - loss: 0.6207 - val_accuracy: 0.7246 - val_loss: 0.5341
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.8005 - loss: 0.4221 - val_accuracy: 0.7213 - val_loss: 0.5489
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.6828 - loss: 0.5997
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.6828 - loss: 0.5997
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step
Precision: 0.6199407699901284
Recall: 0.5825602968460112
Accuracy: 0.6837121212121212
ROC-AUC Score: 0.6680407118032873
Confusion Matrix:
 [[1177  385]
 [ 450  628]]
 Loss: 0.600



changing layers (64) 

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from keras.callbacks import EarlyStopping 
from keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
df= pd.read_csv("Embedded_data.csv")
train = df.drop(['Tokens'], axis = 1)

X = train.Text
Y = train.Label
le = LabelEncoder() 
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
X_train = X_train.astype(str)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = utils.pad_sequences(sequences,maxlen=max_len)

def CNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 1000, input_length=max_len)(inputs)
    layer = Conv1D(filters=512, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Flatten()(layer)
    layer = Dense(64, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

X_test = X_test.astype(str).tolist()

# Now process X_test with the tokenizer and pad_sequences
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences, maxlen=max_len)

# Evaluate the model on the test data
accr = model.evaluate(test_sequences_matrix, Y_test)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

# Predict probabilities using the model
y_pred_proba = model.predict(test_sequences_matrix)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate and print evaluation metrics
print("Precision:", precision_score(Y_test, y_pred))
print("Recall:", recall_score(Y_test, y_pred))
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(Y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, y_pred))


print(' Loss: {:0.3f}\n'.format(accr[0],accr[1]))

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.6629 - loss: 0.6298 - val_accuracy: 0.7032 - val_loss: 0.5691
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.8090 - loss: 0.4004 - val_accuracy: 0.7042 - val_loss: 0.5869
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - accuracy: 0.7377 - loss: 0.5493
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.7377 - loss: 0.5493
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step
Precision: 0.6612576064908722
Recall: 0.6087768440709617
Accuracy: 0.7147727272727272
ROC-AUC Score: 0.6979512008755064
Confusion Matrix:
 [[1235  334]
 [ 419  652]]
 Loss: 0.575



RNN

In [12]:
from keras.layers import Input, Embedding, Dense, Dropout, Activation, LSTM
from keras.optimizers import RMSprop
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(512)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

if isinstance(X_test, pd.Series):
    X_test = X_test.tolist()

# Now process X_test with the tokenizer and pad_sequences
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences, maxlen=max_len)

# Evaluate the model on the test data
accr = model.evaluate(test_sequences_matrix, Y_test)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

# Predict probabilities using the model
y_pred_proba = model.predict(test_sequences_matrix)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate and print evaluation metrics
print("Precision:", precision_score(Y_test, y_pred))
print("Recall:", recall_score(Y_test, y_pred))
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(Y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, y_pred))


print(' Loss: {:0.3f}\n'.format(accr[0],accr[1]))


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, 200, 50)        │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 512)            │     1,153,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_layer (Dense)               │ (None, 1)              │           257 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,334,609 (5.09 MB)

 Trainable params: 1,334,609 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 620s 6s/step - accuracy: 0.5959 - loss: 0.6757 - val_accuracy: 0.6039 - val_loss: 0.6496
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 635s 7s/step - accuracy: 0.6349 - loss: 0.6326 - val_accuracy: 0.6648 - val_loss: 0.6088
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 640s 7s/step - accuracy: 0.6881 - loss: 0.5871 - val_accuracy: 0.6892 - val_loss: 0.5941
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 629s 7s/step - accuracy: 0.7117 - loss: 0.5621 - val_accuracy: 0.6952 - val_loss: 0.5859
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 561s 5s/step - accuracy: 0.7209 - loss: 0.5426 - val_accuracy: 0.6852 - val_loss: 0.5938
83/83 ━━━━━━━━━━━━━━━━━━━━ 51s 607ms/step - accuracy: 0.6909 - loss: 0.5776
83/83 ━━━━━━━━━━━━━━━━━━━━ 50s 603ms/step - accuracy: 0.6909 - loss: 0.5776
83/83 ━━━━━━━━━━━━━━━━━━━━ 48s 549ms/step
Precision: 0.5961857379767828
Recall: 0.6713352007469654
Accuracy: 0.6821969696969697
ROC-AUC Score: 0.680473209041424
Confusion Matrix:
 [[1082  487]
 [ 352  719]]
 Loss:

changing layers (512,128)

In [14]:
from keras.layers import Input, Embedding, Dense, Dropout, Activation, LSTM
from keras.optimizers import RMSprop
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(512)(layer)
    layer = Dense(128,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

if isinstance(X_test, pd.Series):
    X_test = X_test.tolist()

# Now process X_test with the tokenizer and pad_sequences
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences, maxlen=max_len)

# Evaluate the model on the test data
accr = model.evaluate(test_sequences_matrix, Y_test)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

# Predict probabilities using the model
y_pred_proba = model.predict(test_sequences_matrix)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate and print evaluation metrics
print("Precision:", precision_score(Y_test, y_pred))
print("Recall:", recall_score(Y_test, y_pred))
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(Y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, y_pred))


print(' Loss: {:0.3f}\n'.format(accr[0],accr[1]))


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_6 (Embedding)         │ (None, 200, 50)        │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 512)            │     1,153,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out_layer (Dense)               │ (None, 1)              │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,268,817 (4.84 MB)

 Trainable params: 1,268,817 (4.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.5966 - loss: 0.6753 - val_accuracy: 0.6056 - val_loss: 0.6539
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.6320 - loss: 0.6336 - val_accuracy: 0.6708 - val_loss: 0.6125
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.6880 - loss: 0.5942 - val_accuracy: 0.6491 - val_loss: 0.7468
83/83 ━━━━━━━━━━━━━━━━━━━━ 18s 216ms/step - accuracy: 0.6388 - loss: 0.7561
83/83 ━━━━━━━━━━━━━━━━━━━━ 23s 280ms/step - accuracy: 0.6388 - loss: 0.7561
83/83 ━━━━━━━━━━━━━━━━━━━━ 22s 258ms/step
Precision: 0.634090909090909
Recall: 0.2605042016806723
Accuracy: 0.6390151515151515
ROC-AUC Score: 0.5789455361494502
Confusion Matrix:
 [[1408  161]
 [ 792  279]]
 Loss: 0.759

